In [1]:
#Import libraries
import pandas as pd

In [2]:
#!pip install TextBlob

In [3]:
dirPath = r"\data"
reviewPath = r"reviews_Tools_and_Home_Improvement_5.json.gz"
#reviewPath = r"reviews_Musical_Instruments_5.json.gz"
reviewPath = r"cleanData_reviews_Musical_Instruments_5.json.gz"
path = dirPath + "\\" + reviewPath
path = "preprocessed_reviews_Musical_Instruments_5.pkl.gz"
path = "preprocessed_reviews_Tools_and_Home_Improvement_5.pkl.gz"

In [4]:
data = pd.read_pickle(path)
data.shape

(123707, 7)

In [5]:
negativeReviews = data.loc[data['trueRating'] == 0, :]
negativeReviews.shape[0]


10105

In [6]:
#data = data.iloc[0:size*5, :]

In [7]:
negativeReviews.tail(2)

,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
123477,134239,B00JPBDL9W,I have converted three-quarters of my home to ...,"[0, 0]",2.0,0.0,i have convert three quarter of my home to lea...
123556,134322,B00JXSASK2,Amazing that they designed this thing without ...,"[2, 2]",1.0,0.0,amaze that they design this thing without real...


In [8]:
#Pad more negative reviews for better training purposes
testSample = pd.concat([data, negativeReviews])
#testSample = data
#testSample.reset_index(drop=True, inplace=True)
testSample.tail(1)

,masterIndex,asin,reviewText,helpful,overall,trueRating,CleanedJoin
123556,134322,B00JXSASK2,Amazing that they designed this thing without ...,"[2, 2]",1.0,0.0,amaze that they design this thing without real...


In [9]:
import itertools

largeText = testSample.CleanedJoin.to_list()
#largeText = reviewTextProcessing.reviewText.to_list() # Not too good accuracy around 116 missed with %86.94

len(largeText)

133812

In [10]:
#!pip install sklearn


In [11]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [12]:
num_words = 15000

In [13]:
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(largeText)
#tokenizer.word_index

In [14]:
X = largeText
y = testSample['trueRating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=310)

x_train_text = X_train
x_test_text = X_test

x_train_tokens = tokenizer.texts_to_sequences(x_train_text)
x_test_tokens = tokenizer.texts_to_sequences(x_test_text)

In [15]:
len(y_train), len(y_test), len(x_train_text), len(x_test_text)

(100359, 33453, 100359, 33453)

In [16]:
#x_train_tokens[1], x_train_text[1]

In [17]:
import numpy as np

x_train_text[1]
np.array(x_train_tokens[1])
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)


In [18]:
np.mean(num_tokens), np.max(num_tokens)


(111.61465339431442, 4959)

In [19]:
np.std(num_tokens)

133.85901020345165

In [20]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens) 
max_tokens


379

In [21]:
np.sum(num_tokens < max_tokens) / len(num_tokens)


0.9588078797118346

In [22]:
pad = 'pre'
#pad = 'post'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens, padding=pad, truncating=pad)
x_train_pad.shape, x_test_pad.shape


((100359, 379), (33453, 379))

In [23]:
for i in range(3, 4):
    print(x_train_pad[i])


[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [24]:
def tokens_to_string(tokens):
    # Map from tokens back to words.
    words = [inverse_map[token] for token in tokens if token != 0]
    
    # Concatenate all words.
    text = " ".join(words)

    return text

In [25]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [26]:
x_train_text[1]

'very handy and easy to use, these arrive on time and in good condition, nice to have in shop when you need it you need it'

In [27]:
tokens_to_string(x_train_tokens[1])

'very handy and easy to use these arrive on time and in good condition nice to have in shop when you need it you need it'

In [28]:
import tensorflow as tf
tf.__version__

'2.0.0'

In [29]:
#!pip install keras

In [30]:
index = len(x_train_pad)
index
len(y_train), len(y_test), len(y_train) + len(y_test)

(100359, 33453, 133812)

In [31]:
#abstract CNN
import modelCNNImplementation as modelCNN
import importlib
importlib.reload(modelCNN)

Using TensorFlow backend.


<module 'modelCNNImplementation' from '/home/toan/Study/NLP/modelCNNImplementation.py'>

In [32]:
import modelRNNImplementation as modelRNN
import importlib
importlib.reload(modelRNN)

<module 'modelRNNImplementation' from '/home/toan/Study/NLP/modelRNNImplementation.py'>

In [33]:
# keep seed consistently 
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)


In [35]:
from sklearn.metrics import confusion_matrix


In [36]:
# Refactor here to run multiple simulations

cls_true = np.array(y_test)

numOfSimulations = 10

for eachRun in range (0, numOfSimulations):
    model_cnn = modelCNN.compileCNNModel(max_tokens=max_tokens)
    model_rnn = modelRNN.compileRNNModel(max_tokens=max_tokens, embedding_size=200)
    #custom_batch_size=32+eachRun*6
    #custom_batch_size=50+eachRun*6
    custom_batch_size=56
    print(f"batch size {custom_batch_size}")
    #train models
    model_cnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=custom_batch_size)
    #model_rnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=128)
    #predict results
    cnn_y_pred = model_cnn.predict(x=x_test_pad)
    cnn_y_pred = cnn_y_pred.T[0]
    #rnn_y_pred = model_rnn.predict(x=x_test_pad)
    #rnn_y_pred = rnn_y_pred.T[0]
    cnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in cnn_y_pred])
    #rnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in rnn_y_pred])
    #result matrix including duplicated results
    y_test.index
    y_test.values
    result = pd.DataFrame()
    result['Index'] = y_test.index
    result['TrueRating'] = y_test.values
    result['CNNRating'] = cnn_cls_pred
    #result['RNNRating'] = rnn_cls_pred
    # need to drop the duplicates due to negative padding
    result.drop_duplicates(subset="Index", inplace=True)
    #merge results  back
    df = testSample.loc[result.Index.to_list(), :]
    df = df.reset_index()
    df= df.drop_duplicates(subset="index")
    df.rename(columns={"index": "Index"}, inplace=True)
    modelResult = pd.merge(df, result, on="Index", how="inner")
    y_true = modelResult.TrueRating.to_list()
    cnn_pred = modelResult.CNNRating.to_list()
    #rnn_pred = modelResult.RNNRating.to_list() 
    report = dict()
    report['CNN'] = confusion_matrix(y_true, cnn_pred).ravel()
    #report['RNN'] = confusion_matrix(y_true, rnn_pred).ravel()
    y_true = np.array(y_true)
    reportCard = pd.DataFrame(report).T
    reportCard.columns = ['tn', 'fp', 'fn', 'tp']
    reportCard.loc['ActualRating'] = [len(np.where(y_true==0)[0]), 0, 0, len(np.where(y_true==1)[0])]
    reportCard.loc['custom_batch_size'] = custom_batch_size
    reportCard.to_csv(f"run{eachRun}_PaddingNegativeReviews_CNN.csv")
    del model_cnn
    del model_rnn  
    



    
    

batch size 56


/home/toan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 13s 136us/step - loss: 0.2256 - accuracy: 0.9105 - val_loss: 0.1703 - val_accuracy: 0.9299
Epoch 2/3
95341/95341 [==============================] - 13s 141us/step - loss: 0.1252 - accuracy: 0.9526 - val_loss: 0.1524 - val_accuracy: 0.9426
Epoch 3/3
95341/95341 [==============================] - 13s 140us/step - loss: 0.0749 - accuracy: 0.9725 - val_loss: 0.1770 - val_accuracy: 0.9394
batch size 56


/home/toan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 14s 143us/step - loss: 0.2235 - accuracy: 0.9110 - val_loss: 0.1713 - val_accuracy: 0.9314
Epoch 2/3
95341/95341 [==============================] - 14s 144us/step - loss: 0.1230 - accuracy: 0.9532 - val_loss: 0.1404 - val_accuracy: 0.9492
Epoch 3/3
95341/95341 [==============================] - 13s 140us/step - loss: 0.0708 - accuracy: 0.9743 - val_loss: 0.1647 - val_accuracy: 0.9352
batch size 56


...

Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 14s 142us/step - loss: 0.2294 - accuracy: 0.9082 - val_loss: 0.1579 - val_accuracy: 0.9374
Epoch 2/3
95341/95341 [==============================] - 13s 140us/step - loss: 0.1259 - accuracy: 0.9526 - val_loss: 0.1499 - val_accuracy: 0.9406
Epoch 3/3
95341/95341 [==============================] - 13s 139us/step - loss: 0.0721 - accuracy: 0.9733 - val_loss: 0.1273 - val_accuracy: 0.9560
batch size 56


/home/toan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 13s 138us/step - loss: 0.2305 - accuracy: 0.9071 - val_loss: 0.1753 - val_accuracy: 0.9285
Epoch 2/3
95341/95341 [==============================] - 13s 139us/step - loss: 0.1279 - accuracy: 0.9507 - val_loss: 0.1572 - val_accuracy: 0.9400
Epoch 3/3
95341/95341 [==============================] - 13s 140us/step - loss: 0.0781 - accuracy: 0.9712 - val_loss: 0.1380 - val_accuracy: 0.9542
batch size 56


/home/toan/.pyenv/versions/3.7.3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 95341 samples, validate on 5018 samples
Epoch 1/3
95341/95341 [==============================] - 13s 140us/step - loss: 0.2260 - accuracy: 0.9100 - val_loss: 0.1649 - val_accuracy: 0.9352
Epoch 2/3
95341/95341 [==============================] - 13s 140us/step - loss: 0.1226 - accuracy: 0.9535 - val_loss: 0.1447 - val_accuracy: 0.9432
Epoch 3/3
95341/95341 [==============================] - 14s 143us/step - loss: 0.0724 - accuracy: 0.9740 - val_loss: 0.1466 - val_accuracy: 0.9460


In [ ]:
model_cnn = modelCNN.compileCNNModel(max_tokens=max_tokens)
model_cnn.summary()

In [ ]:
model_rnn = modelRNN.compileRNNModel(max_tokens=max_tokens, embedding_size=200)
model_rnn.summary()

In [ ]:
%%time
model_cnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=64)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

In [ ]:
#sess = tf.session(config=tf.ConfigProto(log_device_placement=True))
tf.test.gpu_device_name()


In [ ]:
%%time
model_rnn.fit(x_train_pad, y_train, validation_split=0.05, epochs=3, batch_size=128)
#model.fit(x_train_pad, y_train,validation_split=0.05, epochs=3, batch_size=74)

In [ ]:
#%%time
cnn_result = model_cnn.evaluate(x_test_pad, y_test)

In [ ]:
#%%time
rnn_result = model_rnn.evaluate(x_test_pad, y_test)

In [ ]:
print("Accuracy: {0:.2%}".format(rnn_result[1]))

In [ ]:
print("Accuracy: {0:.2%}".format(cnn_result[1]))


In [ ]:
%%time
cnn_y_pred = model_cnn.predict(x=x_test_pad)
cnn_y_pred = cnn_y_pred.T[0]

In [ ]:
%%time
rnn_y_pred = model_rnn.predict(x=x_test_pad)
rnn_y_pred = rnn_y_pred.T[0]

In [ ]:
pd.DataFrame(cnn_y_pred).describe(), pd.DataFrame(rnn_y_pred).describe()

In [ ]:
cnn_y_pred.shape, rnn_y_pred.shape

In [ ]:
cnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in cnn_y_pred])
rnn_cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in rnn_y_pred])

In [ ]:
len(cnn_cls_pred), len(rnn_cls_pred)

In [ ]:
cnn_accuracy = (cnn_cls_pred==y_test).mean()
rnn_accuracy = (rnn_cls_pred==y_test).mean()
cnn_accuracy, rnn_accuracy

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
#Check for padding
tn, fp, fn, tp = confusion_matrix(y_test, cnn_cls_pred).ravel()
tn, fp, fn, tp

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, rnn_cls_pred).ravel()
tn, fp, fn, tp

In [ ]:
len(np.where(y_test==0)[0]),len(np.where(y_test==1)[0])

In [ ]:
len(np.where(y_train==0)[0]),len(np.where(y_train==1)[0])

In [ ]:
cls_true = np.array(y_test)

In [ ]:
y_test.index
y_test.values
result = pd.DataFrame()
result['Index'] = y_test.index
result['TrueRating'] = y_test.values
result['CNNRating'] = cnn_cls_pred
result['RNNRating'] = rnn_cls_pred

In [ ]:
result.shape

In [ ]:
# need to drop the duplicates due to negative padding
result.drop_duplicates(subset="Index", inplace=True)
result.shape

In [ ]:
result.head(1)

In [ ]:
df = testSample.loc[result.Index.to_list(), :]
df = df.reset_index()
#A.drop_duplicates()
df= df.drop_duplicates(subset="index")
df.rename(columns={"index": "Index"}, inplace=True)
df.shape, df.columns

In [ ]:
df.head(4)

In [ ]:
modelResult = pd.merge(df, result, on="Index", how="inner")
modelResult.head(3)

In [ ]:
#!pip install TextBlob

In [ ]:
import TextBlobImplementation as TextBlob
import importlib
importlib.reload(TextBlob)

In [ ]:
polaritySample = modelResult.iloc[3, :]['CleanedJoin']

TextBlob.getPolarityFromText(polaritySample.split(','))

In [ ]:
import TextPreProcessing as TextProcessing
import importlib
importlib.reload(TextProcessing)

In [ ]:
%%time
modelResult['simplifiedText'] = modelResult.reviewText.apply(TextProcessing.simplifiedTextParagraph)

In [ ]:
#!pip install nltk
#import nltk
#nltk.download('punkt')

In [ ]:
%%time
#not very accurate using stemming
#modelResult['textBlobRating'] = modelResult.CleanedJoin.str.split(',')
modelResult['textBlobRating'] = modelResult.simplifiedText.apply(TextBlob.getPolarityFromText)

In [ ]:
modelResult.head(3)

In [ ]:

y_true = modelResult.TrueRating.to_list()
cnn_pred = modelResult.CNNRating.to_list()
textBlob_pred = modelResult.textBlobRating.to_list()

In [ ]:
from sklearn.metrics import confusion_matrix

y_true = modelResult.TrueRating.to_list()
rnn_pred = modelResult.RNNRating.to_list() 
textBlob_pred = modelResult.textBlobRating.to_list()

In [ ]:
#report = {'CNN': [], 'RNN': None, 'TextBlob':None}
report = dict()

In [ ]:
#tn, fp, fn, tp = confusion_matrix(y_true, cnn_pred).ravel()
report['CNN'] = confusion_matrix(y_true, cnn_pred).ravel()
#tn, fp, fn, tp
report['CNN']

In [ ]:
#tn, fp, fn, tp = confusion_matrix(y_true, rnn_pred).ravel()
report['RNN'] = confusion_matrix(y_true, rnn_pred).ravel()
#tn, fp, fn, tp
report['RNN']

In [ ]:
#tn, fp, fn, tp = confusion_matrix(y_true, textBlob_pred).ravel()
report['TextBlob'] = confusion_matrix(y_true, textBlob_pred).ravel()
#tn, fp, fn, tp

In [ ]:
pd.options.display.max_colwidth = 50

#productsReviewData.cleanedText.head(2)

In [ ]:
y_true = np.array(y_true)

In [ ]:
reportCard = pd.DataFrame(report).T
reportCard.columns = ['tn', 'fp', 'fn', 'tp']

In [ ]:
reportCard.loc['ActualRating'] = [len(np.where(y_true==0)[0]), 0, 0, len(np.where(y_true==1)[0])]
reportCard

In [ ]:
reportCard.to_csv("2nd_PaddingNegativeReviews_RNN_CNN_TextBlob.csv")

## 133K review comparison

## padding extra negative reviews

##### we pick up two unique negative extras with the sampling when we padded

tn, fp, fn, tp

(3913, 554, 742, 27627)  CNN for negative reviews padding 
(3445, 1022, 923, 27446) RNN with negative reviews padding
(1142, 3325, 1782, 26587) textBlob with negative reviews padding 

Note: RNN 3*8 word vector with  batch size = 64

## no padding extra negative reviews

tn, fp, fn, tp

(1667, 916, 813, 27531)  CNN without negative reviews padding
(1034, 1549, 270, 28074) RNN without negative reviews padding
(671, 1912, 1767, 26577) textBlob without negative reviews padding


## Multinomial Naive Bayes no padding extra negative reviews

tn, fp, fn, tp

(509, 2074, 4175, 24169) Multinomial Naive Bayes # n-gram (1,2), num_words = 20000
(26, 83, 393, 1871) Multinomial Naive Bayes # n-gram (1,1), num_words = 3000




## 10K review comparison

## padding extra negative reviews

##### we pick up two unique negative extras with the sampling when we padded

tn, fp, fn, tp

(174, 27, 44, 2214) CNN for negative reviews padding 
(113, 88, 34, 2224)  RNN with negative review padding
(50, 151, 112, 2146) textBlob with negative padding 

## no padding extra negative reviews

tn, fp, fn, tp

(31, 78, 37, 2227)  CNN without negative reviews padding
(3, 106, 2, 2262)   RNN without negative reviews padding
(27, 82, 106, 2158) textBlob without negative reviews padding


## Multinomial Naive Bayes no padding extra negative reviews

tn, fp, fn, tp

(12, 97, 232, 2032) Multinomial Naive Bayes # n-gram (1,2), num_words = 10000
(26, 83, 393, 1871) Multinomial Naive Bayes # n-gram (1,1), num_words = 3000


